In [51]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader(r'data\teste.pdf')
pages = loader.load_and_split()

In [52]:
# Combine as páginas e substitua as tabulações por espaços
text = ' '.join([page.page_content.replace('\t' , ' ') for page in pages])
print(text[0:100])

ESTAÇÃO ZOO - UNIDADE CAMPO BELO
Rua Zacarias de Gois 1288
Parque Colonial, São Paulo/SP - CEP: 0461


In [53]:
import re

def clear_text(text):
    cleaned_text = re.sub(r' +', ' ', text)
    # Remove spaces around hyphens
    cleaned_text = re.sub(r'\s*-\s*', '', cleaned_text)
    # Replace newline characters with spaces
    cleaned_text = cleaned_text.replace('\n', ' ')

    return cleaned_text

In [54]:
clean_text = clear_text(text)

In [55]:
import os 
from langchain import OpenAI

os.environ[ "OPENAI_API_KEY" ] = 'sk-None-p9ffPPgzyXVw4yHAcFOuT3BlbkFJiH7T2cTVFg0pWNEsC7ee'

llm = OpenAI()
tokens = llm.get_num_tokens(clean_text)
print(f"Total de {tokens}")

Total de 549


In [56]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings

text_splitter = SemanticChunker(
    OpenAIEmbeddings(), breakpoint_threshold_type="interquartile"
)

docs = text_splitter.create_documents([clean_text])

In [74]:
type(docs)

list

In [57]:
import openai

def get_embeddings(texts):
    list_embedding = []
    for text in texts:
        parts = [text [i:i + 8192] for i in range(0, len(text), 8192)]
        
        for part in parts:
            response = openai.embeddings.create(
                model="text-embedding-3-small",
                input=part
            )
            list_embedding.append(response.data)

    return list_embedding

embeddings=get_embeddings([doc.page_content for doc in docs])

In [58]:
import pandas as pd 
import numpy as np

content_list = [doc.page_content for doc in docs]

df = pd.DataFrame(content_list, columns=['page_content'])
vectors = [embedding[0].embedding for embedding in embeddings]
array = np.array(vectors)

embeddings_series = pd.Series(list(array))

df['embeddings'] = embeddings_series

In [101]:
embeddings

[[Embedding(embedding=[0.02867904119193554, 0.0039875805377960205, 0.019780201837420464, 0.03791581094264984, -0.032891906797885895, -0.023925485089421272, -0.04654430225491524, 0.05965600907802582, -0.02584042400121689, -0.06573875993490219, 0.03293696418404579, 0.008887574076652527, -0.014733772724866867, -0.030413750559091568, 0.0062686121091246605, 0.004874648526310921, -0.010796881280839443, 0.023497438058257103, -0.02867904119193554, 0.016322046518325806, 0.0034440758172422647, -0.006139072123914957, -0.02638111263513565, -0.03541512414813042, -0.0012996246805414557, -0.010870099999010563, -0.0299631766974926, -0.02408318594098091, 0.04933786392211914, -0.03956040367484093, 0.006415048614144325, -0.014576071873307228, 0.024849161505699158, 0.007012059446424246, -0.03151765838265419, -0.01083630695939064, -0.04266936704516411, -0.032283633947372437, -0.03167535737156868, -0.013190556317567825, -0.01827077940106392, 0.0037538453470915556, 0.049292806535959244, 0.04789602756500244, 

In [61]:
import faiss

array = array.astype('float32')
num_clusters = 50

dimension = array.shape[1]

# Número de pontos de dados
num_points = array.shape[0]

# Verificar se o número de pontos é suficiente para o número de clusters
if num_points < num_clusters:
    num_clusters = num_points
    print(f"Número de clusters ajustado para {num_clusters} devido ao número insuficiente de pontos de dados.")



kmeans = faiss.Kmeans(dimension, num_clusters, niter=20, verbose=True)
kmeans.train(array)

centroids = kmeans.centroids

index = faiss.IndexFlatL2(dimension)
index.add(array)

D, I = index.search(centroids, 1)

Número de clusters ajustado para 2 devido ao número insuficiente de pontos de dados.


In [62]:
D, I = index.search(centroids, 1)

In [63]:
sorted_array = np.sort(I, axis=0)
sorted_array = sorted_array.flatten()
num_docs = len(docs)

valid_indices = [i for i in sorted_array if i < num_docs]
extracted_docs = [docs[i] for i in sorted_array]

In [64]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

model = ChatOpenAI(temperature=0,model="gpt-4")


In [65]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
vc poderia me falar qual o CNPJ e o Advigado desse processo de recuperacao judicial por favor

```{text}```
SUMMARY:
"""
)

In [66]:
chain= (
    prompt
   | model
   |StrOutputParser() )

In [70]:
from tqdm import tqdm


final_summary = ""


for doc in tqdm(extracted_docs, desc="Processing documents"):
    # Get the new summary.
    parts = chunk_text(doc.page_content)

    for part in parts:
        new_summary = chain.invoke({"text": part})
        # Update the list of the last two summaries: remove the first one and add the new one at the end.
        final_summary+=new_summary

Processing documents: 100%|██████████| 2/2 [00:07<00:00,  3.65s/it]


In [72]:
from fpdf import FPDF

class PDF(FPDF):
    def header(self):
        # Select Arial bold 15
        self.set_font('Arial', 'B', 15)
        # Move to the right
        self.cell(80)
        # Framed title
        self.cell(30, 10, 'Summary', 1, 0, 'C')
        # Line break
        self.ln(20)

    def footer(self):
        # Go to 1.5 cm from bottom
        self.set_y(-15)
        # Select Arial italic 8
        self.set_font('Arial', 'I', 8)
        # Page number
        self.cell(0, 10, 'Page %s' % self.page_no(), 0, 0, 'C')

# Instantiate PDF object and add a page
pdf = PDF()
pdf.add_page()
pdf.set_font("Arial", size=12)

# Ensure the 'last_summary' text is treated as UTF-8
# Replace 'last_summary' with your actual text variable if different
# Make sure your text is a utf-8 encoded string
last_summary_utf8 = final_summary.encode('latin-1', 'replace').decode('latin-1')
pdf.multi_cell(0, 10, last_summary_utf8)

# Save the PDF to a file
pdf_output_path = "s_output1.pdf"
pdf.output(pdf_output_path)

C:\Users\israe\AppData\Local\Temp\ipykernel_13488\1849219901.py:6: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  self.set_font('Arial', 'B', 15)
C:\Users\israe\AppData\Local\Temp\ipykernel_13488\1849219901.py:10: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=0 use new_x=XPos.RIGHT, new_y=YPos.TOP.
  self.cell(30, 10, 'Summary', 1, 0, 'C')
C:\Users\israe\AppData\Local\Temp\ipykernel_13488\1849219901.py:25: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font("Arial", size=12)
C:\Users\israe\AppData\Local\Temp\ipykernel_13488\1849219901.py:18: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  self.set_font('Arial', 'I', 8)
C:\Users\israe\AppData\Local\Temp\ipykernel_13488\1849219901.py:20: DeprecationWarnin